In [1]:
%load_ext kedro.ipython

[01/26/25 18:28:17] INFO     Using '/Users/seba/Desktop/With                                        __init__.py:270
                             Jacob/QGraphs/venv/lib/python3.11/site-packages/kedro/framework/projec                
                             t/rich_logging.yml' as logging configuration.                                         

                    INFO     Registered line magic '%reload_kedro'                                   __init__.py:61

                    INFO     Registered line magic '%load_node'                                      __init__.py:63

                    INFO     Resolved project path as: /Users/seba/Desktop/With                     __init__.py:178
                             Jacob/QGraphs/qgraphs.                                                                
                             To set a different path, run '%reload_kedro <project_root>'                           

[01/26/25 18:28:18] INFO     Kedro project qgraphs                                                  __init__.py:144

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:145
                             'pipelines'                                                                           

In [2]:
import networkx as nx

def graph_generator(params):
    node_no = params['node_no']
    edge_p = params['edge_p']
    vers = params['vers']

    g = nx.gnp_random_graph(node_no, edge_p, directed=True)
    unweighted_graph_name = f'digraph_{node_no:03d}_{edge_p:03d}_v{vers:03d}_unw'
    #logger.info(f'Graph created: {unweighted_graph_name}')
    return nx.node_link_data(g, edges="links")

In [3]:
node_no = 16
edge_p = 0.1
vers = 11

In [4]:
g = nx.gnp_random_graph(node_no, edge_p, directed=True)

In [6]:
g = catalog.load('digraph_unw')

[01/10/25 00:37:11] INFO     Loading data from digraph_unw (JSONDataset)...                     data_catalog.py:389

In [7]:
G = nx.node_link_graph(g)

In [8]:
adj_sel = nx.adjacency_matrix(G).toarray()

[01/10/25 00:37:14] WARNING  /var/folders/53/b8z3c5xs0l51w2mzflnyk6400000gn/T/ipykernel_16539/67287 ]8;id=393410;file:///opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=856258;file:///opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#110\110]8;;\
                             9585.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse                 
                             array instead of a matrix in Networkx 3.0.                                            
                               adj_sel = nx.adjacency_matrix(G).toarray()                                          
                                                                                                                   

In [9]:
import numpy as np
def digraph_adj_expand(w_adj):
    exp_adj = w_adj.copy() #.toarray()
    for r in range(w_adj.shape[0]):
        r_sum = np.count_nonzero(w_adj[r])
        if r_sum == 0:
            # No outgoing links - create a loop
            exp_adj[r, r] = 1.0
    return exp_adj

In [10]:
adj_exp = digraph_adj_expand(adj_sel)

In [11]:
adj_exp


array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
def digraph_adj_weigh(unw_adj, params):
    w_adj = unw_adj.copy().astype(float)
    for r in range(unw_adj.shape[0]):
        r_sum = sum(unw_adj[r])
        r_nz = np.count_nonzero(unw_adj[r])
        if r_sum != 0.0:
            # Edges available - generate weights
            if params['method'] == 'rand':
                nz_weights = np.random.random(r_nz)
            else:
                nz_weights = np.array([num*1.0 for num in unw_adj[r] if num])
            nz_weights /= nz_weights.sum()
            w_no = 0
            for c in range(unw_adj.shape[1]):
                if unw_adj[r, c] > 0:
                    w_adj[r, c] = nz_weights[w_no]
                    w_no += 1
    return np.around(w_adj, params['num'])

In [13]:
parameter = catalog.load("params:digraph_adj_weight")

[01/10/25 00:37:25] INFO     Loading data from params:digraph_adj_weight (MemoryDataset)...     data_catalog.py:389

In [14]:
adj_w = digraph_adj_weigh(adj_exp, parameter)

In [15]:
adj_w


array([[0.   , 0.   , 0.   , 0.   , 0.5  , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.5  , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.333, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.333, 0.333, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.5  , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.5  , 0.   ],
       [0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   ,
        0.   , 0.   , 0. 

In [16]:
g_new = nx.DiGraph(adj_w)

In [18]:
nx.node_link_data(g_new)


{
    'directed': True,
    'multigraph': False,
    'graph': {},
    'nodes': [
        {'id': 0},
        {'id': 1},
        {'id': 2},
        {'id': 3},
        {'id': 4},
        {'id': 5},
        {'id': 6},
        {'id': 7},
        {'id': 8},
        {'id': 9},
        {'id': 10},
        {'id': 11},
        {'id': 12},
        {'id': 13},
        {'id': 14},
        {'id': 15}
    ],
    'links': [
        {'weight': 0.5, 'source': 0, 'target': 4},
        {'weight': 0.5, 'source': 0, 'target': 12},
        {'weight': 0.333, 'source': 1, 'target': 3},
        {'weight': 0.333, 'source': 1, 'target': 12},
        {'weight': 0.333, 'source': 1, 'target': 13},
        {'weight': 1.0, 'source': 2, 'target': 11},
        {'weight': 0.5, 'source': 3, 'target': 1},
        {'weight': 0.5, 'source': 3, 'target': 14},
        {'weight': 1.0, 'source': 4, 'target': 3},
        {'weight': 1.0, 'source': 5, 'target': 8},
        {'weight': 1.0, 'source': 6, 'target': 6},
        {'weigh